# Exercise 2 - Mean Average Precision

In [ ]:
import copy
import json
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np

## Objective

In this exercise, you will have to implement the Mean Average Precision (mAP) metric.

In [ ]:
# load data 
with open('data/predictions.json', 'r') as f:
    preds = json.load(f)
    
with open('data/ground_truths.json', 'r') as f:
        gts = json.load(f)

## Details

In [ ]:
### From Udacity's `utils.py`

In [ ]:
def calculate_iou(gt_bbox, pred_bbox):
    """
    calculate iou 
    args:
    - gt_bbox [array]: 1x4 single gt bbox
    - pred_bbox [array]: 1x4 single pred bbox
    returns:
    - iou [float]: iou between 2 bboxes
    """
    xmin = np.max([gt_bbox[0], pred_bbox[0]])
    ymin = np.max([gt_bbox[1], pred_bbox[1]])
    xmax = np.min([gt_bbox[2], pred_bbox[2]])
    ymax = np.min([gt_bbox[3], pred_bbox[3]])
    
    intersection = max(0, xmax - xmin + 1) * max(0, ymax - ymin + 1)
    gt_area = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])
    pred_area = (pred_bbox[2] - pred_bbox[0]) * (pred_bbox[3] - pred_bbox[1])
    
    union = gt_area + pred_area - intersection
    return intersection / union

To do so, you will first have to create the Precision-Recall (PR) curve. Once this curve is created, you need to create the smoothed version as discussed in the lesson. Finally you can use this smoothed version to calculate the mAP.

In [ ]:
# TODO IMPLEMENT THIS SCRIPT

You also have to create a visualization of the PR and smoothed PR curves.

Make sure to check the `Desktop` to see your visualization when running the code.

In [ ]:
### From Udacity's `utils.py`

In [ ]:
def check_results(output):
    round_output = np.round(output * 1e4) / 1e4
    assert round_output == 0.7286, 'Something is wrong with the mAP calculation'
    print('mAP calculation is correct!')

In [ ]:
check_results(mAP)

## Tips

To create the PR curve, you need to sort the predictions based on their confidence score and calculate the precision and recall for each subset of the predictions, as explained in the course.

To make your life easier, you can hard code the smoothed PR curve based on the PR curve, but 
you should think of a scripted version of doing so.